In [1]:
from pathlib import Path
import sys
sys.path.append("../..") # go to parent dir

In [2]:
from aspects.analysis.statistics_bing_liu import (load_all_datasets)

In [62]:
import matplotlib.pyplot as plt
import pandas as pd

from typing import List, Iterable

import spacy

from tqdm import tnrange, tqdm_notebook
import jellyfish
%matplotlib inline

In [46]:
nlp = spacy.load('en')

In [5]:
datasets = load_all_datasets()

In [13]:
# jellyfish.jaro_distance('touchscreen', 'touch screen')
jellyfish.levenshtein_distance('touchscreen', 'touch screen')

1

In [53]:
def clean_aspect_text(aspect_text):
    aspect_text = ' '.join([token.lemma_ for token in nlp(aspect_text)])
    return aspect_text

In [69]:
def get_most_similar_text_pairs(texts: Iterable[str], threshold: int = 0.75):
    similar_pairs = {}
    for text_1 in tqdm(set(texts)):
        for text_2 in set(texts):
            jaro = jellyfish.jaro_winkler(text_1, text_2)
            
            # lemma with spacy
            text_1 = clean_aspect_text(text_1)
            text_2 = clean_aspect_text(text_2)
            
            texts_pair = f'{text_1} ## {text_2}'
            texts_pair_inverse = f'{text_2} ## {text_1}'
            if text_1 != text_2 and jellyfish.jaro_winkler(text_1, text_2) > threshold and texts_pair_inverse not in similar_pairs:
                similar_pairs[texts_pair] = jaro
    return similar_pairs

In [70]:
get_most_similar_text_pairs(['train', 'tarin', 'dev'])

100%|██████████| 3/3 [00:00<00:00,  9.88it/s]


{'tarin ## train': 0.94}

In [ ]:
similar_aspects = {
    dataset_name: get_most_similar_text_pairs(dataset.aspect.tolist())
    for dataset_name, dataset in datasets.items()
}           

  2%|▏         | 5/212 [00:23<16:13,  4.70s/it]

In [ ]:
pd.DataFrame.from_dict(similar_aspects)